In [13]:
import json
import torch
from datasets import Dataset, load_from_disk
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
print(f'CUDA is available: {torch.cuda.is_available()}')

CUDA is available: True


In [30]:
# load scores
with open('llm_scores.jsonl', 'r', encoding='utf-8') as f:
    scores = [json.loads(line) for line in f]

# load extracted texts
with open('texts_extracted.jsonl', 'r', encoding='utf-8') as f:
    texts = [json.loads(line) for line in f]

# combine texts and scores
scored_texts = []
for s in scores:
    idx = s['index']
    score = s['score']
    if not score:
        continue
    text = texts[idx]['extracted_text']
    scored_texts.append({
        'text': text,
        'label': 1 if score >= float(4) else 0
    })

# get sft data for TinyBERT
dataset = Dataset.from_list(scored_texts)
print(f'Total of {len(dataset)} pieces of data labeled.')

Total of 9948 pieces of data labeled.


In [31]:
# Initialize BERT training
bert = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizer.from_pretrained(bert)

def tokenize(sample):
    return tokenizer(
        text=sample['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
    )

dataset = dataset.map(tokenize).shuffle(seed=6)


Map: 100%|██████████| 9948/9948 [00:46<00:00, 215.14 examples/s]


In [32]:
dataset_split = dataset.train_test_split(test_size=0.1, seed=6)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8953
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 995
    })
})

In [33]:
# training setup

model = BertForSequenceClassification.from_pretrained(
    bert,
    num_labels=2  
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./tinybert-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=50,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = logits.argmax(axis=-1)

    acc = accuracy_score(labels, pred)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\linxi\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
trainer.train()
trainer.save_model("tinybert-filter")

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.568100,0.573817,0.719598,0.747487,0.884101,0.810075
2,0.529900,0.545423,0.739698,0.770235,0.876672,0.820014
3,0.498800,0.515048,0.746734,0.808199,0.820208,0.814159
4,0.499600,0.523766,0.740704,0.824726,0.783061,0.803354
5,0.463500,0.527126,0.741709,0.814199,0.800892,0.807491
6,0.451800,0.536603,0.730653,0.838063,0.745914,0.789308
7,0.408000,0.534458,0.746734,0.831496,0.784547,0.807339
8,0.419200,0.533917,0.752764,0.831008,0.796434,0.813354
